In [1]:
import pyspark as ps
from collections import Counter
import pandas as pd
import time
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("nathanscope") 
        .getOrCreate()
        )

In [2]:
sc = spark.sparkContext

In [3]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i 

0: marketplace
1: customer_id
2: review_id
3: product_id
4: product_parent
5: product_title
6: product_category
7: star_rating
8: helpful_votes
9: total_votes
10: vine
11: verified_purchase
12: review_headline
13: review_body
14: review_date


In [4]:
def casting_function(row):
    if row[0] == 'marketplace':
        return ()
    if len(row)!=15:
        return(len(row))
    (marketplace, customer_id, review_id, product_id, product_parent, product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date) = row
    
    
    return (marketplace, str(customer_id), str(review_id), str(product_id), product_parent, product_title,product_category,int(star_rating),int(helpful_votes),int(total_votes),vine,verified_purchase,review_headline,review_body,review_date)




In [5]:
def time_cast(row):
    pattern = '%Y-%m-%d'
    row =list(row)
    row[14] = int(time.mktime(time.strptime(row[14], pattern)))/86400
    return tuple(row)

In [6]:
rdd_books = (sc.textFile('s3://amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz')
            .map(lambda rowstr : rowstr.split("\t"))
            .map(casting_function)
           )

In [7]:
rdd_books =rdd_books.filter(lambda x: len(x)==15).map(time_cast)

In [8]:
pd.DataFrame(rdd_books.take(5))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,US,12076615,RQ58W7SMO911M,0385730586,122662979,Sisterhood of the Traveling Pants (Book 1),Books,4,2,3,N,N,this book was a great learning novel!,this boook was a great one that you could lear...,13070
1,US,12703090,RF6IUKMGL8SF,0811828964,56191234,The Bad Girl's Guide to Getting What You Want,Books,3,5,5,N,N,Fun Fluff,If you are looking for something to stimulate ...,13070
2,US,12257412,R1DOSHH6AI622S,1844161560,253182049,"Eisenhorn (A Warhammer 40,000 Omnibus)",Books,4,1,22,N,N,this isn't a review,never read it-a young relative idicated he lik...,13070
3,US,50732546,RATOTLA3OF70O,0373836635,348672532,Colby Conspiracy (Colby Agency),Books,5,2,2,N,N,fine author on her A-game,Though she is honored to be Chicago Woman of t...,13070
4,US,51964897,R1TNWRKIVHVYOV,0262181533,598678717,The Psychology of Proof: Deductive Reasoning i...,Books,4,0,2,N,N,Execellent cursor examination,Review based on a cursory examination by Unive...,13070


In [9]:
star_counts = rdd_books.map(lambda x: Counter({x[cols['star_rating']]:1})).reduce(lambda x, y: x+y)

In [9]:
day_counts = rdd_books.map(lambda x: Counter({x[cols['review_date']]:1})).collect()

In [ ]:
sum_stars = rdd_books.map(lambda x: x[cols['star_rating']]).reduce(lambda x, y: x+y)

In [ ]:
books_len = rdd_books.count()

In [ ]:
sum_stars/books_len

In [ ]:
books_len

In [ ]:
star_counts

In [ ]:
pd.DataFrame(rdd_books.take(20))

In [ ]:
total_days = day_counts[0]
for item in day_counts:
    total_days = total_days + item

In [ ]:
total_days

In [10]:
day_counts

[Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({13070: 1}),
 Counter({